# Run the Tracker

This jupyter notebook bundles all necessary high-level steps to train the tracker. The first step is to include all necessary modules.

In [1]:
from tqdm import tqdm
import pickle
import sys
from datetime import datetime
sys.path.append("build/")
from cpp_utils import Simulation, ParticleTracker
from time import time, sleep
from python.plotter import Plotter
from python.constants import *
from python.accurateTracker import AccurateTracker
from python.confidentTracker import ConfidentTracker
# from python.particleTracker import ParticleTracker
from python.metrics import Confidence, Accuracy

A simple 2D simulation simulates is used to generate the data of the robot perceptions. The next cells runs a new simulation to generate fresh data. The fresh data is saved by a simulation log pickle file. This cell is optional, we can also load the data from an old simulation log.

In [2]:
T_simulation = 1 * 60  # 10 minutes

sim = Simulation(T_step=0.1, N_humans=3, N_robots=1)
sim_states = []
N_minutes = int(T_simulation / 60)
N_hours = int(T_simulation / 3600)
pbar = tqdm(range(0, N_minutes), desc="Simulation")
for i in pbar:  # always calculate one minute at once to have C++ speedup
    sim_states += sim.step(int(60 / sim.T_step))
    pbar.set_postfix(
        {
            "Simulated time": "{:d}:{:02d} of {:d}:{:02d} hours".format(
                int(i / 60), i % 60, int(T_simulation / 3600), int(T_simulation / 60) % 60
            )
        }
    )

sim_log = {
    "T_step": sim.T_step,
    "T_simulation": T_simulation,
    "N_humans": sim.N_humans,
    "N_robots": sim.N_robots,
    "sim_states": sim_states,
}

filename = os.path.join(LOG_FOLDER, "log_" + datetime.now().strftime("%Y-%m-%d_%H-%M-%S") + ".pkl")
with open(filename, "wb") as outp:
    pickle.dump(sim_log, outp, pickle.HIGHEST_PROTOCOL)

Simulation: 100%|██████████| 1/1 [00:00<00:00, 77.56it/s, Simulated time=0:00 of 0:01 hours]


This cell loads the simulation data from a simulation log. If no fresh data is available, an old simulation log is loaded.

In [3]:
if "sim_log" not in locals():
    filename = os.path.join(LOG_FOLDER, "log_2024-08-30_10-47-52.pkl")
    with open(filename, "rb") as f:
        sim_log = pickle.load(f)
sim_states = sim_log["sim_states"]
T_simulation = sim_log["T_simulation"]
T_step = sim_log["T_step"]
N_humans = sim_log["N_humans"]
N_robots = sim_log["N_robots"]

# load simulation object to have access to member utility functions
sim = Simulation(T_step=T_step, N_humans=N_humans, N_robots=N_robots)

With the simulation data loaded, the actual tracker can be loaded. The tracker is now simulated in another loop over the simulation time. The robot perceptions are just playback from the simulation log. Different Trackers can be tested by creating the tracker object from different classes.

In [4]:
plot = True  # slows down loop significantly!
record_video = False  # slows down loop even more!
if record_video:
    plot = True

confidentTracker = ConfidentTracker(N_robots=N_robots, include_observations=True)
accurateTracker = AccurateTracker(N_robots=N_robots, include_observations=False, train=False)
particleTracker = ParticleTracker(T_step, 3, 1000)

if plot:
    plotter = Plotter(record_frames=record_video)

pbar = tqdm(range(0, len(sim_states)), desc="Simulation")

simulation_time = 0
confidentTracker_node_probabilities = []
accurateTracker_node_probabilities = []
particleTracker_node_probabilities = []
for i in pbar:
    sim_state = sim_states[i]

    # outer list: robots, inner list: perceived humans for every robot
    robot_perceptions = [
        {
            "ego_position": agent["ego_position"],
            "observable_nodes": agent["observable_nodes"],
            "perceived_humans": agent["perceived_humans"],
        }
        for agent in sim_state
        if agent["type"] == "robot"
    ]

    confidentTracker_node_probabilities.append(confidentTracker.add_observation(robot_perceptions))
    _ = confidentTracker.predict()

    accurateTracker_node_probabilities.append(accurateTracker.add_observation(robot_perceptions))
    _ = accurateTracker.predict()

    particleTracker_node_probabilities.append(particleTracker.add_observation(robot_perceptions))
    _ = particleTracker.predict()

    if plot:
        plotter.update(sim_state, particleTracker_node_probabilities[-1])

    pbar.set_postfix(
        {
            "Simulated time": "{:d}:{:02d} of {:d}:{:02d} hours".format(
                int(simulation_time / 3600),
                int(simulation_time / 60) % 60,
                int(T_simulation / 3600),
                int(T_simulation / 60) % 60,
            )
        }
    )

    simulation_time += T_step

if record_video:
    plotter.create_video(T_step)

Simulation:   0%|          | 0/600 [00:00<?, ?it/s]invalid command name "139782253554688process_stream_events"
    while executing
"139782253554688process_stream_events"
    ("after" script)
can't invoke "event" command: application has been destroyed
    while executing
"event generate $w <<ThemeChanged>>"
    (procedure "ttk::ThemeChanged" line 6)
    invoked from within
"ttk::ThemeChanged"
Simulation:   0%|          | 1/600 [00:00<01:07,  8.86it/s, Simulated time=0:00 of 0:01 hours]

0.0014142990112304688
0.001413106918334961


Simulation:   1%|          | 5/600 [00:00<00:46, 12.66it/s, Simulated time=0:00 of 0:01 hours]

0.002438068389892578
0.0016133785247802734
0.001425027847290039
0.001489400863647461


Simulation:   2%|▏         | 9/600 [00:00<00:40, 14.58it/s, Simulated time=0:00 of 0:01 hours]

...

0.00154876708984375
0.0014734268188476562
0.0014743804931640625
0.0014500617980957031


Simulation:  99%|█████████▉| 595/600 [00:35<00:00, 17.01it/s, Simulated time=0:00 of 0:01 hours]

0.0016813278198242188
0.0014495849609375
0.0014560222625732422
0.0015478134155273438


Simulation: 100%|██████████| 600/600 [00:35<00:00, 16.88it/s, Simulated time=0:00 of 0:01 hours]


0.0015075206756591797
0.0016858577728271484
0.001544952392578125


Now the performance of the tracker is evaluated using two performance metrics: Confidence and Accuracy. Both are defined as loss metrics, hence 0 is the optimal value and the larger the value is, the worse the metric is.

In [5]:
confidentTracker_confidence = Confidence(sim_log, confidentTracker_node_probabilities).per_graph()
accurateTracker_confidence = Confidence(sim_log, accurateTracker_node_probabilities).per_graph()
particleTracker_confidence = Confidence(sim_log, particleTracker_node_probabilities).per_graph()

confidentTracker_accuracy = Accuracy(sim_log, confidentTracker_node_probabilities).per_graph()
accurateTracker_accuracy = Accuracy(sim_log, accurateTracker_node_probabilities).per_graph()
particleTracker_accuracy = Accuracy(sim_log, particleTracker_node_probabilities).per_graph()


from python.figures import results_plot

results_plot(
    confidentTracker_confidence=confidentTracker_confidence,
    confidentTracker_accuracy=confidentTracker_accuracy,
    accurateTracker_confidence=accurateTracker_confidence,
    accurateTracker_accuracy=accurateTracker_accuracy,
    particleTracker_confidence=particleTracker_confidence,
    particleTracker_accuracy=particleTracker_accuracy,
)

invalid command name "139782270826944process_stream_events"
    while executing
"139782270826944process_stream_events"
    ("after" script)
can't invoke "event" command: application has been destroyed
    while executing
"event generate $w <<ThemeChanged>>"
    (procedure "ttk::ThemeChanged" line 6)
    invoked from within
"ttk::ThemeChanged"
